In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import cross_val_score

from data_reader import read_data
from data_preprocessing import preprocess_data
from feature_extractor import extract_features

%matplotlib inline

In [38]:
data = read_data()
data = preprocess_data(data, limit=10000)
data.head()

,id,Title,Uri,PublicationDateTimeUTC,ProcedureDisplayName,Amount,RubPrice,CurrencyCode,StatusDisplayName,StatusCode,SuppliersCount,IsWinner,Inn,Kpp,Name,Ogrn,ResultClass
0,0100600000117000008,услуги по капитальному ремонту,https://zakupki.kontur.ru/0100600000117000008,2017-07-27,Закупка у единственного поставщика (подрядчика...,352000.00,352000.00,RUB,Размещение отменено,3.0,0.0,0.0,4720008346,NaN,МЕСТНАЯ АДМИНИСТРАЦИЯ МУНИЦИПАЛЬНОГО ОБРАЗОВАН...,1024702184451,2.0
1,0101100000116000162,Масло сладко-сливочное несоленое Крестьянское,https://zakupki.kontur.ru/0101100000116000162,2016-10-24,Электронный аукцион,3855260.00,3855260.00,RUB,Размещение отменено,3.0,0.0,0.0,0275006455,NaN,УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ИСПОЛНЕНИЯ НАКАЗ...,1020202776714,2.0
2,0101100000117000153,Индивидуальный рацион питания для повседневной...,https://zakupki.kontur.ru/0101100000117000153,2017-09-25,Электронный аукцион,3600000.00,3600000.00,RUB,Размещение отменено,3.0,0.0,0.0,0275006455,NaN,УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ИСПОЛНЕНИЯ НАКАЗ...,1020202776714,2.0
3,0101100000117000154,Консервы мясные,https://zakupki.kontur.ru/0101100000117000154,2017-09-25,Электронный аукцион,15400000.00,15400000.00,RUB,Размещение отменено,3.0,0.0,0.0,0275006455,NaN,УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ИСПОЛНЕНИЯ НАКАЗ...,1020202776714,2.0
4,0101100000416000025,Право заключения государственного контракта на...,https://zakupki.kontur.ru/0101100000416000025,2016-05-30,Открытый конкурс,79196.94,79196.94,RUB,Размещение отменено,3.0,0.0,0.0,0278103383,NaN,УПРАВЛЕНИЕ ФЕДЕРАЛЬНОЙ СЛУЖБЫ ПО НАДЗОРУ В СФЕ...,1040204605154,2.0


In [39]:
# def GetBalancedSample(data, count):
#     cancel_count = min(count // 3, data[data.ResultClass == 2].shape[0])
#     success_count = min((count - cancel_count) // 2, data[data.ResultClass == 1].shape[0])
#     unsuccess_count = count - cancel_count - success_count
#     balanced_sample = pd.concat([data[data.ResultClass == 2][:cancel_count],
#                                  data[data.ResultClass == 1][:success_count],
#                                  data[data.ResultClass == 0][:unsuccess_count]])
#     print(balanced_sample.groupby('ResultClass').size())
#     return balanced_sample

# GetBalancedSample(data, 10000)

In [40]:
data.groupby('ResultClass').size()

ResultClass
0.0    3334
1.0    3333
2.0    3333
dtype: int64

In [41]:
data_head = data
features = extract_features(data_head)[0]

oh CurrencyCode
oh ProcedureDisplayName
oh Ogrn
ng Title
ng ProcedureDisplayName
ng OrgName


In [42]:
features

,RubPrice,ResultClass,Amount,OneHot_CurrencyCode_RUB,OneHot_ProcedureDisplayName_Закрытый аукцион,OneHot_ProcedureDisplayName_Закрытый конкурс,"OneHot_ProcedureDisplayName_Закупка у единственного поставщика (подрядчика, исполнителя)","OneHot_ProcedureDisplayName_Закупка у единственного поставщика (подрядчика, исполнителя) с учетом положений ст. 111 Закона № 44-ФЗ",OneHot_ProcedureDisplayName_Запрос котировок,"OneHot_ProcedureDisplayName_Запрос котировок на выполнение работ по строительству, реконструкции, капитальному ремонту, а также проектных, изыскательских работ в отношении олимпийских объектов и строительству домов взамен земельных участков и объектов недвижимого имущества, изымаемых",...,Ngrams_Name_290,Ngrams_Name_291,Ngrams_Name_292,Ngrams_Name_293,Ngrams_Name_294,Ngrams_Name_295,Ngrams_Name_296,Ngrams_Name_297,Ngrams_Name_298,Ngrams_Name_299
0,352000.00,2.0,352000.00,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3855260.00,2.0,3855260.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3600000.00,2.0,3600000.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15400000.00,2.0,15400000.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,79196.94,2.0,79196.94,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,105270.00,2.0,105270.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1039607.00,2.0,1039607.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,184037.00,2.0,184037.00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,93810.00,2.0,93810.00,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,1544102.57,2.0,1544102.57,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [43]:
features.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(features.drop('ResultClass', axis=1).values,
                            features.ResultClass.values,
                            test_size=0.3,
                            random_state=123456)

In [44]:
# features = pd.DataFrame(features)
# X_train = features.drop(['ResultClass'], 1)
# y_train = features.ResultClass

In [45]:
# rf = RF(n_estimators=10, n_jobs=-1)
# rf = rf.fit(x_train, y_train)
# np.mean(cross_val_score(rf, x_train, y_train))

In [46]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_test, label=y_test, reference=lgb_train, free_raw_data=False)

In [47]:
params = {
        'objective': 'multiclass',
        'metric': 'multi_error',
        'num_classes': 3,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 123456,
        'bagging_fraction': 0.9,
        'bagging_freq': 1,
        'bagging_seed': 123456,
        'max_depth': 8,
        'learning_rate': 0.1,
        'min_data_in_leaf': 11,
        'num_iteration': 100,
        'num_threads': 2
    }

In [48]:
gbm = lgb.train(params,
                    lgb_train,
                    valid_sets=[lgb_train, lgb_eval])  # eval training data
#                     feature_name=features.drop('ResultClass', axis=1).columns.values)

gbm.save_model(f'result.txt')

C:\Users\user\Anaconda3\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's multi_error: 0.0795714	valid_1's multi_error: 0.0963333
[2]	training's multi_error: 0.0697143	valid_1's multi_error: 0.0926667
[3]	training's multi_error: 0.0658571	valid_1's multi_error: 0.0866667
[4]	training's multi_error: 0.0688571	valid_1's multi_error: 0.09
[5]	training's multi_error: 0.0674286	valid_1's multi_error: 0.0896667
[6]	training's multi_error: 0.0645714	valid_1's multi_error: 0.0863333
[7]	training's multi_error: 0.0645714	valid_1's multi_error: 0.0886667
[8]	training's multi_error: 0.0625714	valid_1's multi_error: 0.0883333
[9]	training's multi_error: 0.0618571	valid_1's multi_error: 0.09
[10]	training's multi_error: 0.0605714	valid_1's multi_error: 0.088
[11]	training's multi_error: 0.0597143	valid_1's multi_error: 0.0883333
[12]	training's multi_error: 0.0574286	valid_1's multi_error: 0.0873333
[13]	training's multi_error: 0.0564286	valid_1's multi_error: 0.0866667
[14]	training's multi_error: 0.0548571	valid_1's multi_error: 0.0886667
[15]	training's

In [49]:
num_round = 12
lgb.cv(params, lgb_train, num_round, nfold=5)

{'multi_error-mean': [0.099138025603603713,
  0.095142000610934502,
  0.090711180931734844,
  0.092140976414344711,
  0.092569650830682998,
  0.089139547258784674,
  0.089283424228170788,
  0.088997302798455857,
  0.089139241354526583,
  0.089996385085681599,
  0.088282302287887068,
  0.087282097623371552],
 'multi_error-stdv': [0.0089689718160436763,
  0.0070932746865404748,
  0.0063917624658708502,
  0.0054280916849138864,
  0.0042116228830246039,
  0.0058275338029563023,
  0.0054957639791477454,
  0.0076966715008743919,
  0.0077623678711720746,
  0.0084377142916973701,
  0.0086273385696262923,
  0.0085155361725776507]}

In [50]:
y_pred = gbm.predict(X_test)

In [51]:
test = pd.DataFrame(y_test, columns=['test'])
test['prediction'] = [np.argmax(x) for x in y_pred]

In [52]:
test['is_same'] = test.apply(lambda row: int(row[0] == row[1]), axis=1)
f = {'is_same': ['sum', 'size', 'mean']}
groups = test.groupby('test').agg(f)
groups['is_same'].sort_values('mean', ascending=False)

,sum,size,mean
test,,,
2.0,967,1025,0.943415
0.0,908,988,0.919028
1.0,891,987,0.902736


In [ ]:
# len([x == y for x, y in zip(y_test, predicted)])

In [ ]:
# result = pd.DataFrame(y_test, columns=['test'])
# result['pred'] = predicted

In [ ]:
# result.head()

In [ ]:
# result['is_same'] = result.apply(lambda row: int(row[0] == row[1]), axis=1)

In [ ]:
# result.head()

In [ ]:
# accuracy = sum(result.is_same) / result.shape[0]
# accuracy

In [ ]:
# result.groupby('test').agg({'is_same': ['sum', 'size', 'mean']})['is_same'].sort_values('mean', ascending=False)

In [ ]:
# table = pd.crosstab(predicted, y_test, colnames=['Actual Results'], rownames=['Predicted Results'])
# table

In [ ]:
# recall = {}
# precision = {}
# for column in table.columns:
#     recall[column] = table[column][column] / table.apply(lambda row: sum(row), axis=0)[column]
#     precision[column] = table[column][column] / table.apply(lambda row: sum(row), axis=1)[column]
# recall

In [ ]:
# precision

In [ ]:
# F = {}
# for column in table.columns:
#     F[column] = 2 * (recall[column] * precision[column]) / (recall[column] + precision[column])
# F